<a href="https://colab.research.google.com/github/antoniocfetngnu/InteligArtificial1/blob/main/2doParcial/2Parcial_4_en_raya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/juansensio/axr/blob/master/axr/00_intro.ipynb)

# Aprendizaje por Refuerzo 4 en raya
Para transformar el codigo en 4 en raya tenemos que hacer los siguientes cambios:



*   Primero, como es 4 en raya, necesitamos cambiar el tamañó del tablero a 4x4
*   Luego tenemos que cambiar las condiciones de victoria para que coincidan con el nuevo tablero, diagonales de 4 casillas y rectas de 4 casillas.
*   Finalmente la representación del estado del tablero se cambió para manejar un tablero de 4x4 (16 posiciones en lugar de 9).






In [1]:
import numpy as np


class Board():
    def __init__(self):
        self.state = np.zeros((4, 4))

    def valid_moves(self):
        return [(i, j) for j in range(4) for i in range(4) if self.state[i, j] == 0]

    def update(self, symbol, row, col):
        if self.state[row, col] == 0:
            self.state[row, col] = symbol
        else:
            raise ValueError("¡Movimiento ilegal!")

    def is_game_over(self):
        # comprobar filas y columnas
        for i in range(4):
            for j in range(1):
                if np.abs(np.sum(self.state[i, j:j+4])) == 4:
                    return np.sign(np.sum(self.state[i, j:j+4]))
                if np.abs(np.sum(self.state[j:j+4, i])) == 4:
                    return np.sign(np.sum(self.state[j:j+4, i]))
        # comprobar diagonales
        for i in range(1):
            for j in range(1):
                diag1 = np.sum([self.state[i+k, j+k] for k in range(4)])
                diag2 = np.sum([self.state[i+3-k, j+k] for k in range(4)])
                if np.abs(diag1) == 4:
                    return np.sign(diag1)
                if np.abs(diag2) == 4:
                    return np.sign(diag2)
        # empate
        if len(self.valid_moves()) == 0:
            return 0
        # seguir jugando
        return None

    def reset(self):
        self.state = np.zeros((4, 4))


In [2]:
from tqdm import tqdm

class Game():
    def __init__(self, player1, player2):
        player1.symbol = 1
        player2.symbol = -1
        self.players = [player1, player2]
        self.board = Board()

    def selfplay(self, rounds=100):
        wins = [0, 0]
        for i in tqdm(range(1, rounds + 1)):
            self.board.reset()
            for player in self.players:
                player.reset()
            game_over = False
            while not game_over:
                for player in self.players:
                    action = player.move(self.board)
                    self.board.update(player.symbol, action[0], action[1])
                    for player in self.players:
                        player.update(self.board)
                    if self.board.is_game_over() is not None:
                        game_over = True
                        break
            self.reward()
            for ix, player in enumerate(self.players):
                if self.board.is_game_over() == player.symbol:
                    wins[ix] += 1
        return wins


    def reward(self):
        winner = self.board.is_game_over()
        if winner == 0:  # empate
            for player in self.players:
                player.reward(0.5)
        else:  # le damos 1 recompensa al jugador que gana
            for player in self.players:
                if winner == player.symbol:
                    player.reward(1)
                else:
                    player.reward(0)

In [3]:
class Agent():
    def __init__(self, alpha=0.5, prob_exp=0.5):
        self.value_function = {}  # tabla con pares estado -> valor
        self.alpha = alpha  # learning rate
        self.positions = []  # guardamos todas las posiciones de la partida
        self.prob_exp = prob_exp  # probabilidad de explorar

    def reset(self):
        self.positions = []

    def move(self, board, explore=True):
        valid_moves = board.valid_moves()
        # exploracion
        if explore and np.random.uniform(0, 1) < self.prob_exp:
            # vamos a una posición aleatoria
            ix = np.random.choice(len(valid_moves))
            return valid_moves[ix]
        # explotacion
        # vamos a la posición con más valor
        max_value = -1000
        for row, col in valid_moves:
            next_board = board.state.copy()
            next_board[row, col] = self.symbol
            next_state = str(next_board.reshape(4*4))
            value = 0 if self.value_function.get(next_state) is None else self.value_function.get(next_state)
            if value >= max_value:
                max_value = value
                best_row, best_col = row, col
        return best_row, best_col

    def update(self, board):
        self.positions.append(str(board.state.reshape(4*4)))

    def reward(self, reward):
        # al final de la partida (cuando recibimos la recompensa)
        # iteramos por todos los estados actualizando su valor en la tabla
        for p in reversed(self.positions):
            if self.value_function.get(p) is None:
                self.value_function[p] = 0
            self.value_function[p] += self.alpha * (reward - self.value_function[p])
            reward = self.value_function[p]

In [4]:
agent1 = Agent(prob_exp=0.5)
agent2 = Agent()

game = Game(agent1, agent2)

game.selfplay(100000)

100%|██████████| 100000/100000 [28:07<00:00, 59.25it/s]


[33918, 28551]

In [5]:
import pandas as pd

funcion_de_valor = sorted(agent1.value_function.items(), key=lambda kv: kv[1], reverse=True)
tabla = pd.DataFrame({'estado': [x[0] for x in funcion_de_valor], 'valor': [x[1] for x in funcion_de_valor]})

tabla

,estado,valor
0,[ 0. 0. 0. 1. -1. 0. 0. 1. 0. -1. -1. ...,1.0
1,[ 0. 0. 0. 1. 0. 0. 0. 1. -1. -1. -1. ...,1.0
2,[ 0. 0. 0. 1. 0. 0. -1. 1. 0. -1. 0. ...,1.0
3,[ 0. 0. 0. 1. 0. 0. 0. 1. 0. -1. -1. ...,1.0
4,[ 0. 0. 0. 0. 0. 0. 0. 0. 0. -1. -1. -...,1.0
...,...,...
531981,[ 1. 0. 0. 1. -1. 0. -1. -1. -1. -1. 1. ...,0.0
531982,[-1. -1. 0. -1. 1. 0. -1. -1. 1. -1. 1. ...,0.0
531983,[ 1. 1. 1. -1. -1. 1. -1. 1. 1. 0. -1. -...,0.0
531984,[ 1. 0. 1. -1. -1. 1. -1. 1. 1. 0. -1. -...,0.0


In [6]:
import pickle

with open('agente.pickle', 'wb') as handle:
    pickle.dump(agent1.value_function, handle, protocol=pickle.HIGHEST_PROTOCOL)